### Import the Libraries

In [8]:
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
from sklearn.model_selection import train_test_split

### Import the clean data

In [39]:
df_clean = pd.read_csv('/content/clean_fraud_data.csv')

In [40]:
df_clean.head(5)

,Unnamed: 0,amt,merch_lat,merch_long,is_fraud,age,gender_cat,category_food_dining,category_gas_transport,category_grocery_net,...,job_Tree surgeon,job_Video editor,job_Volunteer coordinator,job_Warden/ranger,job_Waste management officer,job_Water engineer,job_Water quality scientist,job_Web designer,job_Wellsite geologist,job_Writer
0,11984,6.54,37.058647,-83.598917,0.0,60,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,97462,181.67,36.384228,-91.163671,0.0,52,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,22029,38.97,46.316279,-90.054852,0.0,23,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,60654,19.26,38.869364,-85.898899,0.0,29,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,66865,115.92,42.315847,-72.061026,0.0,60,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
X = df_clean.drop(['is_fraud'],axis=1)
y = df_clean.is_fraud

In [49]:
X.isnull().sum()

AttributeError: 'Tensor' object has no attribute 'isnull'

In [42]:
# convert the panda dataframe into pytorch tensors
X = torch.tensor(X.values, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32).reshape(-1,1)

In [43]:
# Split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=42)

In [44]:
X_train.shape

torch.Size([1779, 452])

### Building Classification Model

In [45]:
class Deep(nn.Module):
  def __init__(self):
    super().__init__()
    self.layer1 = nn.Linear(452, 60)
    self.act1 = nn.ReLU()
    self.layer2 = nn.Linear(60, 60)
    self.act2 = nn.ReLU()
    self.layer3 = nn.Linear(60,60)
    self.act3 = nn.ReLU()
    self.output = nn.Linear(60,1)
    self.sigmoid = nn.Sigmoid()


  def forward(self, x):
    x = self.act1(self.layer1(x))
    x = self.act2(self.layer2(x))
    x = self.act3(self.layer3(x))
    x = self.sigmoid(self.output(x))
    return x

### Train the Model

In [65]:
import torch.optim as optim
import tqdm
import copy


def model_train(model, X_train, y_train, X_test, y_test):
  # loss function and optimizer
  loss_fn = nn.BCELoss()
  optimizer = optim.Adam(model.parameters(), lr=0.0001)

  n_epochs = 50 #number of epoch to run
  batch_size = 10
  batches_per_epoch = len(X_train)

  best_acc = - np.inf
  best_weights = None

  train_loss_hist = []
  train_acc_hist = []
  test_loss_hist = []
  test_acc_hist = []

  for epoch in range(n_epochs):
    epoch_loss = []
    epoch_acc = []
    model.train()
    with tqdm.trange(batches_per_epoch, unit='batch',mininterval=0) as bar:
      bar.set_description(f"Epoch {epoch}")
      for start in bar:
        X_batch = X_train[start:start+batch_size]
        y_batch = y_train[start:start+batch_size]

        # forward pass
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)

        # backward pass
        optimizer.zero_grad()
        loss.backward()

        # update weight
        optimizer.step()

        # print progress
        acc = ( y_pred.round() == y_batch ).float().mean()
        epoch_loss.append(float(loss))
        epoch_acc.append(float(acc))
        bar.set_postfix(
            loss = float(loss),
            acc=float(acc)
        )
    model.eval()
    y_pred = model(X_test)
    loss = loss_fn(y_pred, y_test)
    acc = ( y_pred.round() == y_test ).float().mean()
    acc = float(acc)
    train_loss_hist.append(np.mean(epoch_loss))
    train_acc_hist.append(np.mean(epoch_acc))
    test_loss_hist.append(float(loss))
    test_acc_hist.append(float(acc))
    if acc > best_acc:
      best_acc = acc
      best_weights = copy.deepcopy(model.state_dict())
    print(f"Epoch {epoch} validation: Cross-entropy={loss}, Accuracy={acc}")

  model.load_state_dict(best_weights)
  return best_acc



In [66]:
model = Deep()
acc = model_train(model, X_train, y_train, X_test, y_test)

Epoch 0: 100%|██████████| 1779/1779 [00:13<00:00, 129.49batch/s, acc=1, loss=9.06e-6]


Epoch 0 validation: Cross-entropy=2.350877523422241, Accuracy=0.8833552002906799


Epoch 1: 100%|██████████| 1779/1779 [00:15<00:00, 116.05batch/s, acc=1, loss=0.00923]


Epoch 1 validation: Cross-entropy=0.8949822783470154, Accuracy=0.9174311757087708


Epoch 2: 100%|██████████| 1779/1779 [00:13<00:00, 131.89batch/s, acc=1, loss=9.81e-5]


...

Epoch 47 validation: Cross-entropy=0.4240530729293823, Accuracy=0.9134993553161621


Epoch 48: 100%|██████████| 1779/1779 [00:14<00:00, 119.64batch/s, acc=1, loss=0.163]


Epoch 48 validation: Cross-entropy=0.30921670794487, Accuracy=0.9174311757087708


Epoch 49: 100%|██████████| 1779/1779 [00:15<00:00, 116.68batch/s, acc=1, loss=0.16]


Epoch 49 validation: Cross-entropy=0.5229389071464539, Accuracy=0.9213630557060242


In [60]:
print("Accuracy (wide): %.2f" % acc)

Accuracy (wide): 0.92


In [ ]:
import matplotlib.pyplot as plt

plt.plot(train_loss_hist, label="train")
plt.plot(test_loss_hist, label="test")
plt.xlabel("epochs")
plt.ylabel("cross entropy")
plt.legend()
plt.show()

plt.plot(train_acc_hist, label="train")
plt.plot(test_acc_hist, label="test")
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.legend()
plt.show()